In [49]:
import json
import requests
from requests_oauthlib import OAuth1
import json
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import tweepy
import seaborn as sns
%matplotlib inline
import os
import pandas as pd
import seaborn as sns
plt.style.use('seaborn')

### Settings

In [81]:

raw_data_dir = "../../data/raw_data/iu-bloomington/twitter/"
raw_file = "indiana-university-bl_twitter_data.csv"

processed_data_dir =  "../../data/processed_data/iu-bloomington/twitter/"
file = "indiana-university-bl_twitter_data.xls"


## Most frequent words
most_frequent_words = "indiana-university-bl_twitter_data_most_frequent_words.csv"
most_frequent_hashtags = "indiana-university-bl_twitter_data_most_frequent_hashtags.csv"
most_frequent_screennames = "indiana-university-bl_twitter_data_most_frequent_screennames.csv"

In [89]:
## read data
import csv
df = pd.read_csv(os.path.join(raw_data_dir,raw_file))

## save currentfile 
df.to_csv(os.path.join(processed_data_dir,raw_file),quoting=csv.QUOTE_ALL)

In [90]:
df.head()

,created,time,text,polarity,subjectivity,tweet_id,truncated,user,user_followers_count,user_friends_count,...,retweet_count,favorite_count,reply_count,at_mentions,hashtags,urls,source,lat,long,tweet_type
0,2019-06-13,20:08:46,RT @IndianaUniv: New degrees requested @IUBloo...,0.136364,0.454545,1139263360636215297,False,IUBloomington,214859,703,...,3,0,0,"IndianaUniv, IUBloomington",NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,retweet
1,2019-06-13,20:08:21,RT @LunaLightxx: I just made my fall schedule ...,0.000000,0.000000,1139263255053029376,False,IUBloomington,214859,703,...,1,0,0,LunaLightxx,NaN,NaN,"<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,NaN,retweet
2,2019-06-13,20:04:19,"Just a few blocks from the Sample Gates, you'l...",0.050000,0.216667,1139262243332349952,True,IUBloomington,214859,703,...,5,86,0,NaN,NaN,https://t.co/3BEtHdERz1,"<a href=""https://sproutsocial.com"" rel=""nofoll...",NaN,NaN,tweet
3,2019-06-13,17:58:10,RT @IndianaUniv: Welcome to @IUFortWayne for t...,0.800000,0.900000,1139230494678274049,False,IUBloomington,214859,703,...,1,0,0,"IndianaUniv, IUFortWayne",IUBOT,NaN,"<a href=""http://twitter.com/download/iphone"" r...",NaN,NaN,retweet
4,2019-06-13,15:06:01,Big Red 200 will support research in:\n♦ artif...,-0.200000,0.366667,1139187172983095298,True,IUBloomington,214859,703,...,10,41,0,NaN,NaN,https://t.co/Z9Yc5EurkA,"<a href=""https://sproutsocial.com"" rel=""nofoll...",NaN,NaN,quote


## Most Frequent Words


In [67]:
from nltk.tokenize import TweetTokenizer
from nltk.stem.porter import PorterStemmer

def tweet_tokenizer(txt):
    try:
        tokenizer = TweetTokenizer()
        all_tokens = tokenizer.tokenize(txt.lower())
        # this line filters out all tokens that are entirely non-alphabetic characters
        filtered_tokens = [t for t in all_tokens if t.islower()]
        # filter out all tokens that are <=2 chars
        filtered_tokens = [x for x in filtered_tokens if len(x)>2]
    except IndexError:
        filtered_tokens = []
    return(filtered_tokens)


tweets_text = str(df["text"])

In [68]:
tweet_tokenizer(tweets_text)

['@indianauniv',
 'new',
 'degrees',
 'requested',
 '@iubloo',
 '@lunalightxx',
 'just',
 'made',
 'fall',
 'schedule',
 'just',
 'few',
 'blocks',
 'from',
 'the',
 'sample',
 'gates',
 "you'l",
 '@indianauniv',
 'welcome',
 '@iufortwayne',
 'for',
 'big',
 'red',
 'will',
 'support',
 'research',
 'artif',
 'makin',
 'our',
 'way',
 'downtown',
 'walking',
 'fast',
 'faces',
 'https://t.co/xpw04zqe9t',
 '@jessicaabriscoe',
 'survived',
 'day',
 '@iubl',
 "there's",
 'much',
 'around',
 'bloomington',
 'just',
 '@btnlivebig',
 'with',
 'grant',
 'from',
 'the',
 '@nih',
 'summertime',
 'the',
 'perfect',
 'time',
 'explore',
 'bloo',
 '@iubarchives',
 'june',
 'theodore',
 'roose',
 'bloomington',
 'has',
 'been',
 'named',
 '@campuspride',
 'bloomington',
 'has',
 'been',
 'named',
 '@campuspride',
 '@iulifesci',
 'congrats',
 '@iubloomington',
 'soci',
 '@iuadmissions',
 'our',
 'vote',
 'for',
 '@iubloomingt',
 'meet',
 'heather',
 'heather',
 'has',
 'cool',
 'job',
 'the',
 'trea

In [69]:
## Get most frequent words
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')

def get_most_frequent_words(text_series, stop_words = None, ngram_range = (1,2)):
    count_vectorizer = CountVectorizer(analyzer = "word",
                                       tokenizer = tweet_tokenizer,
                                       stop_words = stop_words,
                                       ngram_range = ngram_range 
                                      )
    
    term_freq_matrix = count_vectorizer.fit_transform(text_series)
    terms = count_vectorizer.get_feature_names()
    term_frequencies = term_freq_matrix.sum(axis = 0).tolist()[0]

    term_freq_df = (pd.DataFrame(list(zip(terms, term_frequencies)), columns = ["token","count"])
                    .set_index("token")
                    .sort_values("count",ascending = False))
    return term_freq_df


stop_words =   nltk.corpus.stopwords.words('english') + [
 '.',
 ',',
 '--',
 '\'s',
 '?',
 ')',
 '(',
 ':',
 '\'',
 '\'re',
 '"',
 '-',
 '}',
 '{',
 u'—',
 ]

term_freq_df = get_most_frequent_words(df["text"],stop_words = stop_words)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mchivuku/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
term_freq_df.to_csv(os.path.join(processed_data_dir,most_frequent_words))

### Most Frequent Hashtags and Screennames

In [71]:
df.columns

Index(['created', 'time', 'text', 'polarity', 'subjectivity', 'tweet_id',
       'truncated', 'user', 'user_followers_count', 'user_friends_count',
       'user_favourites_count', 'user_statuses_count', 'retweeted_status',
       'month', 'year', 'retweet_count', 'favorite_count', 'reply_count',
       'at_mentions', 'hashtags', 'urls', 'source', 'lat', 'long',
       'tweet_type'],
      dtype='object')

In [72]:
hashtags = []
screen_names = []
for index, row in df.iterrows():
    if isinstance(row["hashtags"], str):
        hashtags.extend([t for t in str(row["hashtags"]).split(", ")])
    if isinstance(row["at_mentions"], str):
        screen_names.extend([t for t in row["at_mentions"].split(", ")]) 
    

In [73]:
print("Hashtags: ", len(hashtags),"Screen_names:", len(screen_names))

Hashtags:  1411 Screen_names: 6064


In [74]:
from prettytable import PrettyTable
from collections import Counter
for label, data in (('Screen Name', screen_names),
                    ('Hashtag', hashtags)):
    pt = PrettyTable(field_names=[label, 'Count'])
    c = Counter(data)
    
    cf = pd.DataFrame.from_dict(c, orient='index').reset_index()
    if label == "Screen Name":
        cf = cf.rename(columns={'index':'screen_name', 0:'count'}).sort_values("count",ascending = False)
        cf.to_csv(os.path.join(processed_data_dir,most_frequent_hashtags))
    
    else:
        cf = cf.rename(columns= {'index':"hashtag",0:'count'}).sort_values("count",ascending = False)
        cf.to_csv(os.path.join(processed_data_dir,most_frequent_screennames))

    

In [76]:
cf.head()

,hashtag,count
14,GoIU,62
50,Little500,53
71,iubloomington,43
54,IUBloomington,38
3,IU,25
